In [10]:
import importlib

import numpy as np
import pandas as pd
import numpy
import lab_graph

from json import loads, dumps

importlib.reload(lab_graph)

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

df = pd.read_csv("data/all-colors-unique.csv")

#Create a 2D array of Lab values
lab_list = list(df["L*a*b Value"])
lab_list = [x.split(", ") for x in lab_list]
lab_points = [[float(x) for x in row] for row in lab_list]

lab_graph.generate_lab_3d_all_colors(df, "All colors", "part_2/plots/All_Points_350x350")

In [3]:
from scipy.spatial import ConvexHull

hull = ConvexHull(points=lab_points)

hull.volume

476503.15507333336

In [13]:
convex_df = df.iloc[hull.vertices].copy()

result = convex_df.to_json(orient="records")
print(result)
#lab_graph.generate_lab_3d_all_colors(convex_df, "Convex Hull of Points", "N/a")

[{"Color Name":"Berry","PMS Color Code":"233CP","L*a*b Value":"45.38, 71.84, -7.30","Hex":"C6057B","L":45.38,"a":71.84,"b":-7.3},{"Color Name":"Charity Pink","PMS Color Code":"212CP","L*a*b Value":"57.92, 59.63, -3.98","Hex":"DE5795","L":57.92,"a":59.63,"b":-3.98},{"Color Name":"Deep Teal","PMS Color Code":"3025CP","L*a*b Value":"32.45, -23.19, -27.24","Hex":"005876","L":32.45,"a":-23.19,"b":-27.24},{"Color Name":"Forest","PMS Color Code":"5535 C","L*a*b Value":"17.54, -12.05, 1.20","Hex":"183029","L":17.54,"a":-12.05,"b":1.2},{"Color Name":"Gold","PMS Color Code":"137C","L*a*b Value":"76.47, 31.14, 81.64","Hex":"FFA400","L":76.47,"a":31.14,"b":81.64},{"Color Name":"Maize Yellow","PMS Color Code":"7548CP","L*a*b Value":"83.14, 5.01, 85.02","Hex":"F2CA00","L":83.14,"a":5.01,"b":85.02},{"Color Name":"Navy","PMS Color Code":"533C","L*a*b Value":"16.99, 2.57, -18.63","Hex":"202A44","L":16.99,"a":2.57,"b":-18.63},{"Color Name":"Olive","PMS Color Code":"5747CP","L*a*b Value":"31.03, -6.88, 2

In [ ]:
from scipy.spatial.distance import pdist, squareform

euclidean_distances = squareform(pdist(lab_points))

convex_hull_score = lab_graph.score(euclidean_distances, list(hull.vertices))
print(convex_hull_score)

In [ ]:
import plotly.graph_objects as go

def generate_lab_3d_all_colors_hull(df, figure_title, output_filename, hull_df):
    # Parse the L*a*b values from the string format
    df[['L', 'a', 'b']] = df['L*a*b Value'].str.split(', ', expand=True).astype(float)

    hull_df[['L', 'a', 'b']] = hull_df['L*a*b Value'].str.split(', ', expand=True).astype(float)
    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=df['a'],        # a* axis (green-red)
        y=df['L'],        # L* axis (lightness)
        z=df['b'],        # b* axis (blue-yellow)
        mode='markers+text',
        #text=df['Color Name'],
        hovertemplate=
            '<b>%{text}</b><br>' +
            'L*: %{y:.1f}<br>' +
            'a*: %{x:.1f}<br>' +
            'b*: %{z:.1f}<br>',
        marker=dict(
            size=7,
            color=['#' + hex.strip() for hex in df['Hex']],  # Use actual hex colors
            opacity=0.5
        ),
        textposition='top center',
        textfont=dict(
            size=8
        )
    )])

    #See if this works
    fig.add_scatter3d(
        x=convex_df['a'],
        y=convex_df['L'],
        z=convex_df['b'],
        mode='markers+text',
        text=convex_df['Color Name'],
        hovertemplate=
            '<b>%{text}</b><br>' +
            'L*: %{y:.1f}<br>' +
            'a*: %{x:.1f}<br>' +
            'b*: %{z:.1f}<br>',
        marker=dict(
            size=7,
            color=['#' + hex.strip() for hex in convex_df['Hex']],  # Use actual hex colors
            opacity=1.0
        ),
        textposition='top center',
        textfont=dict(
            size=8
        )
    )

    # Update the layout
    fig.update_layout(
        template="plotly",
        title=figure_title,
        scene=dict(
            # TODO Adjust before publish
            aspectratio=dict(x=1, y=1, z=1),
            xaxis_title='a* (green to red)',
            yaxis_title='L* (black to white)',
            zaxis_title='b* (blue to yellow)',
            # Set appropriate ranges for each axis
            # Originally -128, 128
            xaxis=dict(range=[-128, 128]),
            # Originally [0, 100]
            yaxis=dict(range=[0, 100]),
            zaxis=dict(range=[-128, 128])
        ),
        #TODO Change for Export
        width=351,
        height=351,
        showlegend=False,
        margin=dict(l=1, r=1, t=1, b=1)
    )

    fig.add_trace(go.Mesh3d(
        x=hull_df['a'],        # a* axis (green-red)
        y=hull_df['L'],        # L* axis (lightness)
        z=hull_df['b'],
        hoverinfo='skip',
        color="white",
        opacity=0.5,
        alphahull=0
    ))

    # Add better camera angle
    fig.update_layout(scene_camera=dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.5, y=1.5, z=1.5)
    ))

    # Show the plot
    #fig.show()

    # Optionally save to HTML file
    fig.write_html(output_filename + ".html")

generate_lab_3d_all_colors_hull(df, "All colors", "part_2/plots/All_Points_Convex_Hull_350x350", hull_df=convex_df)


In [ ]:
#Want to remove the 5 closest points

selected = [lab_points[i] for i in hull.vertices]

distances = squareform(pdist(selected))

#The distances should be recomputed every time/exclude chosen distances but we will ignore for now
avg_distance = []
for item in distances:
    mysum = item.sum()
    avg_distance += [mysum/len(distances)]

small = []

for i in range(5):
    smallest = min(avg_distance)
    smallest_index = avg_distance.index(smallest)
    small += [smallest_index]
    avg_distance[smallest_index] = 200

#small is the index we want to remove
hull_indices = list(hull.vertices)

for item in small:
    hull_indices.pop(item)

convex_hull_score_16 = lab_graph.score(euclidean_distances, hull_indices)

print(convex_hull_score_16)


In [ ]:
convex_df_16 = df.iloc[hull_indices].copy()

lab_graph.generate_lab_3d_all_colors(convex_df_16, "Optimal 16 colors", "N/a")

## Convex Hull Scores

21 Colors: 82.8510 (Slightly outperforms the greedy search)

16 Colors: 84.3717 (Also outperforms greedy search)

# K-Means Clustering

In [ ]:
from scipy.cluster.vq import kmeans

centroids = kmeans(lab_points, 16)

In [ ]:
close_to_centroids = []

from scipy.spatial import distance
closest_indices = []
for centroid in centroids[0]:

    current_distances = []

    #For each color in the space, compute the distance
    for color in lab_points:
        current_distances += [distance.euclidean(centroid, color)]

    #The index of the minimum distance is the color we want
    closest = min(current_distances)
    closest_index = current_distances.index(closest)
    closest_indices += [closest_index]

len(closest_indices)

In [ ]:
centroid_df = df.iloc[closest_indices].copy()

lab_graph.generate_lab_3d_all_colors(centroid_df, "K-Means Points", "N/a")

In [ ]:

# kmeans_selected = [lab_points[i] for i in closest_indices]
# print(kmeans_selected)
score = lab_graph.score(euclidean_distances, closest_indices)
print(score)

## K means Scores

Score for 24 Points: 62.5369
Score for 16 Points: 56.8744